# Backend switches and conversion time

Measure backend switch counts and total conversion time for a circuit requiring backend changes.

In [ ]:
import pandas as pd
from quasar.circuit import Circuit, Gate
from quasar.simulation_engine import SimulationEngine
from quasar.cost import Backend
from quasar.planner import PlanResult, PlanStep

def measure_switches_conversion_time() -> pd.DataFrame:
    circuit = Circuit([
        Gate('H', [0]), Gate('CX', [0, 1]), Gate('T', [1]),
        Gate('CX', [1, 2]), Gate('H', [2]),
        Gate('CX', [2, 3]), Gate('T', [3]), Gate('H', [3]),
    ])
    steps = [
        PlanStep(0, 2, Backend.TABLEAU),
        PlanStep(2, 5, Backend.MPS),
        PlanStep(5, 7, Backend.DECISION_DIAGRAM),
        PlanStep(7, 8, Backend.TABLEAU),
    ]
    plan = PlanResult(table=[], final_backend=None, gates=circuit.gates, explicit_steps=steps)
    plan.explicit_conversions = []
    engine = SimulationEngine()
    engine.scheduler.run(circuit, plan, instrument=True)
    _, metrics = engine.scheduler.run(circuit, plan, instrument=True)
    return pd.DataFrame([
        {
            'backend_switches': metrics.backend_switches,
            'conversion_time': sum(metrics.conversion_durations),
        }
    ])

measure_switches_conversion_time()
